Just some more problems.

> https://leetcode.com/problems/flatten-a-multilevel-doubly-linked-list/

In [32]:
class Node:
    def __init__(self, val, prev, next, child):
        self.val = val
        self.prev = prev
        self.next = next
        self.child = child
        
def flatten(head):
    return _flatten(head)[0]

def _flatten(head):
    if head is None:
        return None, None
    
    out = curr = head
    while curr.child is None and curr.next is not None:
        curr = curr.next
    
    if curr.child is not None:
        first, last = _flatten(curr.child)
        curr.child = None
        if curr.next is not None:
            curr.next.prev, last.next = last, curr.next
        curr.next, first.prev = first, curr
        last.next, curr = _flatten(last.next)
    
    return head, curr

In [28]:
root = Node(1, None, None, None)

root.next = Node(2, root, None, None)

root.next.next = Node(5, root.next, None, None)

root.next.child = Node(3, root.next, None, None)

root.next.child.next = Node(4, root.next.child, None, None)

In [29]:
result = flatten(root)

In [30]:
flatten(result)

* This is correct, modulo an edge case.
* The solutions point out that there is a stack-based solution, can we figure that out?

In [46]:
class Node:
    def __init__(self, val, prev, next, child):
        self.val = val
        self.prev = prev
        self.next = next
        self.child = child
        
def flatten(head):
    if head is None: return head
    
    stack = [head]
    head = Node(0, None, None, None)
    curr = head
    
    while stack:
        node = stack.pop()
        curr.next, node.prev = node, curr
        curr = curr.next
        
        if node.next is not None:
            stack.append(node.next)
        if node.child is not None:
            stack.append(node.child)
            node.child = None
    
    head = head.next
    head.prev = None
    return head

In [47]:
root = Node(1, None, None, None)

root.next = Node(2, root, None, None)

root.next.next = Node(5, root.next, None, None)

root.next.child = Node(3, root.next, None, None)

root.next.child.next = Node(4, root.next.child, None, None)

In [48]:
result = flatten(root)

* Yep. :) This is an easier-to-write and cleaner solution to this problem. Lesson learned for the future, consider cases in which stacks are useful!

> https://leetcode.com/problems/recover-binary-search-tree

* I did not find the tree relationship necessary to solve this problem because I am not sufficiently familiar with the tree structure. After looking up and coming to understand the relationship, I nevertheless struggled with the implementation because I did not understand in-order traversal well enough.

  In particular, I did not realize that in-order traversal has the important property that nodes are visited in-order with respect to their value in the binary tree.
  
  Although I know how to write an in-order traversal algorithm, this problem requires a version of this algorithm that keeps a reference to the node immediately prior to the current one in traversal order. Getting this structure took a lot of figuring out. See `inOrderTraversalWithPreviousRef`.
  
  Clearly my understanding of tree search is insufficient for the demands!

In [26]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

In [33]:
def inOrderTraversal(root):
    if root.left:
        inOrderTraversal(root.left)
    # workon node
    print(root.val)
    if root.right:
        inOrderTraversal(root.right)

In [37]:
root = TreeNode(3)
root.left = TreeNode(1)
root.left.right= TreeNode(2)

In [38]:
inOrderTraversal(root)

1
2
3


In [43]:
def inOrderTraversalWithParentRef(root, parent=None):
    if root.left:
        inOrderTraversalWithParentRef(root.left, root)
    print(root.val, parent.val if parent else None)
    if root.right:
        inOrderTraversalWithParentRef(root.right, root)

In [44]:
inOrderTraversalWithParentRef(root)

1 3
2 1
3 None


In [132]:
def inOrderTraversalWithPreviousRef(root, previous=None):
    if root.left:
        previous = inOrderTraversalWithPreviousRef(root.left, previous)
    
    if previous is not None:
        print(root.val, previous.val)
    else:
        print(root.val, previous)
    
    if root.right:
        last = inOrderTraversalWithPreviousRef(root.right, root)
    else:
        last = root
    
    return last

In [133]:
root = TreeNode(3)
root.left = TreeNode(1)
root.left.right= TreeNode(2)
inOrderTraversalWithPreviousRef(root)

1 None
2 1
3 2


OK now we modify this traversal algorithm to do the work required of it by our algorithm.

In [162]:
def recoverTree(root):
    first = second = None
    
    def traverse(root, first, second, previous=None):
        if root.left:
            first, second, previous = traverse(root.left, first, second, previous)

        print(root.val, previous.val if previous else None)
        if previous is not None:
            if first is None and previous.val > root.val:
                first = previous
            elif second is None and previous.val > root.val:
                second = root

        if root.right:
            first, second, last = traverse(root.right, first, second, root)
        else:
            last = root
        return first, second, last
        
    a, b, _ = traverse(root, None, None, None)
    a.val, b.val = b.val, a.val
    return root

In [155]:
root = TreeNode(1)
root.left = TreeNode(3)
root.left.right= TreeNode(2)
result = recoverTree(root)

> <ipython-input-154-233e875db2cf>(22)recoverTree()
-> a.val, b.val = b.val, a.val
(Pdb) a.val, b.val
root = <__main__.TreeNode object at 0x10f65bdd8>
(Pdb) !a.val, b.val
(3, 1)
(Pdb) root.val
1
(Pdb) root.left.val
3
(Pdb) root.left.rigt.val
*** AttributeError: 'TreeNode' object has no attribute 'rigt'
(Pdb) root.left.right.val
2
(Pdb) c


In [160]:
result.val, result.left.val, result.left.right.val

(3, 1, 2)

In [163]:
root = TreeNode(3)
root.left = TreeNode(1)
root.right= TreeNode(4)
root.right.left = TreeNode(2)
result = recoverTree(root)

1 None
3 1
2 3
4 2


AttributeError: 'NoneType' object has no attribute 'val'

* It doesn't work because the relationship defined in the answer doesn't always hold? Whatever, spent enough time on this one as it is.

> https://leetcode.com/problems/merge-intervals/

In [184]:
def merge(intervals):
    if len(intervals) == 0: return intervals
    
    intervals = sorted(intervals, key=lambda intvl: intvl[0])
    out = intervals[:1]
    current_l, current_r = out[-1][0], out[-1][1]
    
    for (l, r) in intervals[1:]:
        if l <= current_r:
            current_l = min(current_l, l)
            current_r = max(current_r, r)
            out[-1] = [current_l, current_r]
        else:
            current_l, current_r = l, r
            out.append([l, r])
    
    return out

In [185]:
merge([[1,2], [2,3]])

[[1, 3]]

In [186]:
merge([[1,2], [2,3], [3, 4]])

[[1, 4]]

In [187]:
merge([[1,2], [2,3], [3, 4], [5, 6]])

[[1, 4], [5, 6]]

In [188]:
merge([[1,2], [1,2], [1,2], [1,3], [1,4], [5, 6]])

[[1, 4], [5, 6]]

In [190]:
merge([])

[]

In [193]:
merge([[500, 1250], [1000, 1500], [3, 10]])

[[3, 10], [500, 1500]]

* Easy solve, didn't particularly learn anything. $O(n\log{n})$.